In [1]:
import torch
from torch import autograd
from torchvision import datasets,transforms
import torch.utils.data as data
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.utils as nn_utils
import numpy as np
import torch.nn.functional as F

In [2]:
class PosTaggerData(data.Dataset):
    def __init__(self, training_data):
        self.training_data = training_data
        self.word_to_ix = {"null":0}
        self.tag_to_ix = {"null":0}
        for sent, tags,length in self.training_data:
            for word in sent:
                if word not in self.word_to_ix:
                    self.word_to_ix[word] = len(self.word_to_ix)
            for word in tags:
                if word not in self.tag_to_ix:
                    self.tag_to_ix[word] = len(self.tag_to_ix)
        #print(self.word_to_ix)
        #print(self.tag_to_ix)

        self.gen_all_data()

    def __len__(self):
        return len(self.items)

    def __getitem__(self,idx):
        return self.items[idx]

    def gen_all_data(self):
        self.items = []
        for sent,tags,length in self.training_data:
            data = self.prepare_sequence(sent,self.word_to_ix)
            targets = self.prepare_sequence(tags,self.tag_to_ix)
            #print('data:',data)
            #print('targets:',targets)

            self.items.append((data,targets,length))

    def prepare_sequence(self,seq, to_ix):
        idxs = [to_ix[w] for w in seq]
        while len(idxs) != 50:
            idxs.append(0)
        tensor = torch.LongTensor(idxs)
        return tensor

In [3]:
mydata = []
with open('../process/new_crf_testdata','r') as f:
    lines = f.read().split('\n')
t_sen, t_tag = [], []
for i in range(len(lines)):
    line = lines[i].split(' ')
    if len(line) < 2:
        mydata.append((t_sen, t_tag, len(t_sen)))
        t_sen, t_tag = [], []
        continue
    t_sen.append(line[0])
    t_tag.append(line[1] + line[2])
    

In [4]:
len(mydata)

13408

In [5]:
tagger = PosTaggerData(mydata)

In [6]:
BATCH_SIZE = 4
class LSTMPosTagger(nn.Module):
    def __init__(self, vocab_size, tag_size):
        super(LSTMPosTagger,self).__init__()

        self.vocab_size = vocab_size
        self.embedding_size = 100
        self.target_size = tag_size
        self.lstm_hidden_size = 100
        self.bilstm = nn.LSTM(input_size = 100, hidden_size = self.lstm_hidden_size, bidirectional = True)
        self.word_embedding = nn.Embedding(self.vocab_size,self.embedding_size)#词嵌入层 input:seq_len,N , output:seqlen,N,embedding_size
        self.fully = nn.Linear(2*self.lstm_hidden_size,self.target_size)


    def forward(self,input,length_list):
        embedded = self.word_embedding(input)
        lstm_out,h_c = self.bilstm(embedded)  #4*50*200
        tag_space = self.fully(lstm_out.view(-1, 200))
        tag_score = F.log_softmax(tag_space)
        return tag_score
      

In [7]:
trainloader = torch.utils.data.DataLoader(
                    tagger.items, 
                    batch_size=4,
                    shuffle=True, 
                    num_workers=2)

In [8]:
len(tagger.items)

13408

In [10]:
from sklearn.model_selection import train_test_split
import time
cur_max_acc = 0.9227
model = LSTMPosTagger(len(tagger.word_to_ix), len(tagger.tag_to_ix))
optimizer = optim.SGD(model.parameters(), lr=0.01)
#optimizer = optim.Adagrad(model.parameters(), lr=0.01)
loss_function = nn.NLLLoss()
x_data = [(tagger.items[i][0], tagger.items[i][2]) for i in range(len(tagger.items))]
y_data = [tagger.items[i][1] for i in range(len(tagger.items))]
train_x, test_x, train_y, test_y = train_test_split(x_data, y_data, test_size=0.1, random_state=42)
trainset = [(train_x[i][0],train_x[i][1], train_y[i]) for i in range(len(train_x))]
testset = [(test_x[i][0], test_x[i][1],test_y[i]) for i in range(len(test_x))]
trainloader = torch.utils.data.DataLoader(
                    trainset, 
                    batch_size=4,
                    shuffle=True, 
                    num_workers=2)
testloader = torch.utils.data.DataLoader(
                    testset, 
                    batch_size=4,
                    shuffle=False, 
                    num_workers=2)


for epoch in range(50):  

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        sen, length, target = data
        sen = Variable(torch.from_numpy(np.array(sen)))
        target = Variable(torch.from_numpy(np.array(target)))
        length = Variable(torch.from_numpy(np.array(length)))
        model = torch.load('models/model1544536900.pkl')
        tag_scores = model(sen, length)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()

        loss = loss_function(tag_scores, target.reshape(-1))
        #loss = criterion(outputs, labels)
        loss.backward()   

        # 更新参数 
        optimizer.step()

        # 打印log信息
        # loss 是一个scalar,需要使用loss.item()来获取数值，不能使用loss[0]
        running_loss += loss.item()
        if i % 200 == 199: # 每200个batch打印一下训练状态
            print('[%d, %5d] loss: %.3f' \
                  % (epoch+1, i+1, running_loss / 200))
            running_loss = 0.0
    with torch.no_grad():
        correct, total = 0, 0
        for data in testloader:
            sen, length, target = data
            sen = Variable(torch.from_numpy(np.array(sen)))
            target = Variable(torch.from_numpy(np.array(target)))
            length = Variable(torch.from_numpy(np.array(length)))
            tag_scores = model(sen, length)
            tag_scores = tag_scores.reshape((-1, 50, 1210))
            _, predicted = torch.max(tag_scores, 2)
            for kk in range(len(target)):
                for jj in range(len(target[kk])):
                    if target[kk][jj] == 0:break
                    if target[kk][jj] == predicted[kk][jj]:
                        correct += 1
                    total += 1
            #res = res + [map_[int(predicted[i])] for i in range(len(predicted))]
        if cur_max_acc < float(100 * correct / total):
            print('-------------cur max is ::::----------'+str(float(100 * correct / total)))
            cur_max_acc = float(100 * correct / total)
            torch.save(model, 'models/model' + str(int(time.time())) + '.pkl')  # 保存整个网络
        print('result of epoch ' + str(epoch) + 'accuracy is : %d %%' % (100 * correct / total))
                

/home/works/tools/anaconda2/envs/python3/lib/python3.7/site-packages/ipykernel/__main__.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


[1,   200] loss: 0.210
[1,   400] loss: 0.208
[1,   600] loss: 0.217
[1,   800] loss: 0.219
[1,  1000] loss: 0.202
[1,  1200] loss: 0.214
[1,  1400] loss: 0.199
[1,  1600] loss: 0.213
[1,  1800] loss: 0.193
[1,  2000] loss: 0.195
[1,  2200] loss: 0.196
[1,  2400] loss: 0.221
[1,  2600] loss: 0.187
[1,  2800] loss: 0.214
[1,  3000] loss: 0.214
-------------cur max is ::::----------92.27118179951167
result of epoch 0accuracy is : 92 %


/home/works/tools/anaconda2/envs/python3/lib/python3.7/site-packages/torch/serialization.py:250: UserWarning: Couldn't retrieve source code for container of type LSTMPosTagger. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


[2,   200] loss: 0.200
[2,   400] loss: 0.177
[2,   600] loss: 0.219
[2,   800] loss: 0.200
[2,  1000] loss: 0.189
[2,  1200] loss: 0.215
[2,  1400] loss: 0.212
[2,  1600] loss: 0.218
[2,  1800] loss: 0.204
[2,  2000] loss: 0.210
[2,  2200] loss: 0.204
[2,  2400] loss: 0.216
[2,  2600] loss: 0.202
[2,  2800] loss: 0.230
[2,  3000] loss: 0.215
result of epoch 1accuracy is : 92 %
[3,   200] loss: 0.202
[3,   400] loss: 0.228
[3,   600] loss: 0.197
[3,   800] loss: 0.215
[3,  1000] loss: 0.209
[3,  1200] loss: 0.213
[3,  1400] loss: 0.187
[3,  1600] loss: 0.187
[3,  1800] loss: 0.217
[3,  2000] loss: 0.205
[3,  2200] loss: 0.218
[3,  2400] loss: 0.201
[3,  2600] loss: 0.178
[3,  2800] loss: 0.226
[3,  3000] loss: 0.241
result of epoch 2accuracy is : 92 %
[4,   200] loss: 0.224
[4,   400] loss: 0.196
[4,   600] loss: 0.204
[4,   800] loss: 0.210
[4,  1000] loss: 0.196
[4,  1200] loss: 0.198
[4,  1400] loss: 0.223
[4,  1600] loss: 0.223
[4,  1800] loss: 0.200
[4,  2000] loss: 0.223
[4,  220

[24,   200] loss: 0.182
[24,   400] loss: 0.231
[24,   600] loss: 0.205
[24,   800] loss: 0.191
[24,  1000] loss: 0.202
[24,  1200] loss: 0.192
[24,  1400] loss: 0.209
[24,  1600] loss: 0.224
[24,  1800] loss: 0.191
[24,  2000] loss: 0.219
[24,  2200] loss: 0.221
[24,  2400] loss: 0.233
[24,  2600] loss: 0.194
[24,  2800] loss: 0.214
[24,  3000] loss: 0.200
result of epoch 23accuracy is : 92 %
[25,   200] loss: 0.202
[25,   400] loss: 0.216
[25,   600] loss: 0.219
[25,   800] loss: 0.202
[25,  1000] loss: 0.203
[25,  1200] loss: 0.223
[25,  1400] loss: 0.199
[25,  1600] loss: 0.211
[25,  1800] loss: 0.231
[25,  2000] loss: 0.220
[25,  2200] loss: 0.183
[25,  2400] loss: 0.203
[25,  2600] loss: 0.195
[25,  2800] loss: 0.197
[25,  3000] loss: 0.210
result of epoch 24accuracy is : 92 %
[26,   200] loss: 0.209
[26,   400] loss: 0.219
[26,   600] loss: 0.210
[26,   800] loss: 0.225
[26,  1000] loss: 0.195
[26,  1200] loss: 0.192
[26,  1400] loss: 0.186
[26,  1600] loss: 0.209
[26,  1800] lo

[44,  2400] loss: 0.245
[44,  2600] loss: 0.222
[44,  2800] loss: 0.197
[44,  3000] loss: 0.199
result of epoch 43accuracy is : 92 %
[45,   200] loss: 0.193
[45,   400] loss: 0.217
[45,   600] loss: 0.205
[45,   800] loss: 0.206
[45,  1000] loss: 0.223
[45,  1200] loss: 0.212
[45,  1400] loss: 0.208
[45,  1600] loss: 0.194
[45,  1800] loss: 0.216
[45,  2000] loss: 0.197
[45,  2200] loss: 0.174
[45,  2400] loss: 0.220
[45,  2600] loss: 0.205
[45,  2800] loss: 0.217
[45,  3000] loss: 0.229
result of epoch 44accuracy is : 92 %
[46,   200] loss: 0.201
[46,   400] loss: 0.192
[46,   600] loss: 0.215
[46,   800] loss: 0.211
[46,  1000] loss: 0.214
[46,  1200] loss: 0.205
[46,  1400] loss: 0.187
[46,  1600] loss: 0.240
[46,  1800] loss: 0.194
[46,  2000] loss: 0.238
[46,  2200] loss: 0.214
[46,  2400] loss: 0.183
[46,  2600] loss: 0.232
[46,  2800] loss: 0.221
[46,  3000] loss: 0.187
result of epoch 45accuracy is : 92 %
[47,   200] loss: 0.179
[47,   400] loss: 0.209
[47,   600] loss: 0.222
[

Traceback (most recent call last):
  File "/home/works/tools/anaconda2/envs/python3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/works/tools/anaconda2/envs/python3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/works/tools/anaconda2/envs/python3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/works/tools/anaconda2/envs/python3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/works/tools/anaconda2/envs/python3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/works/tools/anaconda2/envs/python3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  Fi

KeyboardInterrupt: 

In [18]:
ix_to_word, ix_to_tag = {}, {}
for key in tagger.word_to_ix:
    ix_to_word[tagger.word_to_ix[key]] = key
for key in tagger.tag_to_ix:
    ix_to_tag[tagger.tag_to_ix[key]] = key

In [26]:
tagger.word_to_ix['null']

0

In [43]:
from sklearn.model_selection import train_test_split
import time
cur_max_acc = 0.9227
model = LSTMPosTagger(len(tagger.word_to_ix), len(tagger.tag_to_ix))
optimizer = optim.SGD(model.parameters(), lr=0.01)
#optimizer = optim.Adagrad(model.parameters(), lr=0.01)
loss_function = nn.NLLLoss()
x_data = [(tagger.items[i][0], tagger.items[i][2]) for i in range(len(tagger.items))]
y_data = [tagger.items[i][1] for i in range(len(tagger.items))]
train_x, test_x, train_y, test_y = train_test_split(x_data, y_data, test_size=0.1, random_state=42)
trainset = [(train_x[i][0],train_x[i][1], train_y[i]) for i in range(len(train_x))]
testset = [(test_x[i][0], test_x[i][1],test_y[i]) for i in range(len(test_x))]
trainloader = torch.utils.data.DataLoader(
                    trainset, 
                    batch_size=4,
                    shuffle=True, 
                    num_workers=2)
testloader = torch.utils.data.DataLoader(
                    testset, 
                    batch_size=4,
                    shuffle=False, 
                    num_workers=2)


with torch.no_grad():
    correct, total = 0, 0
    t_pred, t_real, t_text = [], [], []
    for data in testloader:
        sen, length, target = data
        sen = Variable(torch.from_numpy(np.array(sen)))
        target = Variable(torch.from_numpy(np.array(target)))
        length = Variable(torch.from_numpy(np.array(length)))
        model = torch.load('models/model1544583165.pkl')
        tag_scores = model(sen, length)
        tag_scores = tag_scores.reshape((-1, 50, 1210))
        _, predicted = torch.max(tag_scores, 2)
        for l in range(len(predicted)):
            pred, real, text = [], [], []
            for u in range(len(predicted[l])):
                if sen[l][u]!=0:
                    text.append(ix_to_word[int(sen[l][u])])
                if predicted[l][u] != 0:
                    pred.append(ix_to_tag[int(predicted[l][u])])
                if target[l][u] != 0:
                    total += 1
                    real.append(ix_to_tag[int(target[l][u])])
                if target[l][u] != predicted[l][u]:
                    pred[-1] = pred[-1]#'+++' + pred[-1] + '+++'
                    real[-1] = real[-1]#'+++' + real[-1] + '+++'
                else:
                    if target[l][u] != 0 and predicted[l][u] != 0:
                        correct += 1
            t_pred.append(' '.join(pred))
            t_real.append(' '.join(real))
            t_text.append(' '.join(text))
    print('result accuracy is : %d %%' % (100 * correct / total))


/home/works/tools/anaconda2/envs/python3/lib/python3.7/site-packages/ipykernel/__main__.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


result accuracy is : 92 %


In [47]:
count = 0
for i in range(len(t_text)):
    if t_real[i] != t_pred[i]:
        count += 1
        real = t_real[i].split()
        pred = t_pred[i].split()
        text = t_text[i].split()
        for h in range(len(real)):
            if real[h] != pred[h]:
                print(''.join(text))
                if h > 0 and h < len(real) - 1:
                    print(text[h-1],text[h],text[h+1],real[h], pred[h])
                elif h > 0:
                    print(text[h-1],text[h],real[h], pred[h])
                elif h < len(real) - 1:
                    print(text[h],text[h+1],real[h], pred[h])
print(1.0 * count / len(t_text))

这样的修养的演员即使正走红也不过如过眼云烟而已成不了真正意义上的艺术家
也 不 过 bu2 bu4
这样的修养的演员即使正走红也不过如过眼云烟而已成不了真正意义上的艺术家
成 不 了 bu5 bu4
这样的修养的演员即使正走红也不过如过眼云烟而已成不了真正意义上的艺术家
不 了 真 liao3 le5
因此加速农村中医药适用人才的培养是加强农村中医药工作的一个战略重点
的 一 个 yi2 yi1
特别对因牙槽骨萎缩牙床低平全口假牙戴不住者帮助最为明显
戴 不 住 bu2 bu4
我们愿与广大读者一起耕耘教育这片沃土共享芬芳四溢的花香收获累累硕果
者 一 起 yi4 yi1
我们愿与广大读者一起耕耘教育这片沃土共享芬芳四溢的花香收获累累硕果
教 育 这 yu5 yu4
我们愿与广大读者一起耕耘教育这片沃土共享芬芳四溢的花香收获累累硕果
获 累 累 lei2 lei4
我们愿与广大读者一起耕耘教育这片沃土共享芬芳四溢的花香收获累累硕果
累 累 硕 lei3 lei4
国防大学政委王茂润中将与延安八一敬老院八十五岁的老红军柴福耀亲切交谈
中 将 与 jiang4 jiang1
苏州本地的庙观就很多定慧寺玄妙观桃花庵还有那令历代文人牵肠挂肚的寒山寺
庙 观 就 guan4 guan1
苏州本地的庙观就很多定慧寺玄妙观桃花庵还有那令历代文人牵肠挂肚的寒山寺
妙 观 桃 guan4 guan1
苏州本地的庙观就很多定慧寺玄妙观桃花庵还有那令历代文人牵肠挂肚的寒山寺
花 庵 还 an1 cai4
通讯员卜凡广最大的感受是团长今年更忙了在团部没呆几天总往边防钻井点跑
往 边 防 bian1 bian5
通讯员卜凡广最大的感受是团长今年更忙了在团部没呆几天总往边防钻井点跑
防 钻 井 zuan4 zuan1
红娘子插金簪戴银花比牡丹芍药胜五倍从容出阁宛若云母天仙
娘 子 插 zi3 zi5
红娘子插金簪戴银花比牡丹芍药胜五倍从容出阁宛若云母天仙
芍 药 胜 yao5 yao4
上次苑梅到同学家去伍太太叫她顺便弯到荀家去送个信也是免得让荀太太又给酒钱
太 太 叫 tai5 tai4
上次苑梅到同学家去伍太太叫她顺便弯到荀家去送个信也是免得让荀太太又给酒钱
免 得 让 de5 de2
上次苑梅到同学家去伍太太叫她顺便弯到荀家去送个信也是免得让荀太太又给酒钱
太 太 又 tai5 tai4

设 计 的 ji5 ji4
工人一按开关电动螺丝刀就嗡嗡地响了起来嗡地一声螺帽就按设计的力拧紧了
力 拧 紧 ning2 ning3
夏夜在场院乘凉闻着浓烈的旱烟味儿听着粗豪沉哑的话语如走入童话境界
在 场 院 chang2 chang3
夏夜在场院乘凉闻着浓烈的旱烟味儿听着粗豪沉哑的话语如走入童话境界
儿 听 着 er5 ting1
夏夜在场院乘凉闻着浓烈的旱烟味儿听着粗豪沉哑的话语如走入童话境界
听 着 粗 ting1 zhe5
夏夜在场院乘凉闻着浓烈的旱烟味儿听着粗豪沉哑的话语如走入童话境界
着 粗 豪 zhe5 cu1
夏夜在场院乘凉闻着浓烈的旱烟味儿听着粗豪沉哑的话语如走入童话境界
粗 豪 沉 cu1 hao2
夏夜在场院乘凉闻着浓烈的旱烟味儿听着粗豪沉哑的话语如走入童话境界
豪 沉 哑 hao2 chen2
夏夜在场院乘凉闻着浓烈的旱烟味儿听着粗豪沉哑的话语如走入童话境界
哑 的 话 ya3 de5
夏夜在场院乘凉闻着浓烈的旱烟味儿听着粗豪沉哑的话语如走入童话境界
的 话 语 de5 hua4
夏夜在场院乘凉闻着浓烈的旱烟味儿听着粗豪沉哑的话语如走入童话境界
话 语 如 hua4 yu3
夏夜在场院乘凉闻着浓烈的旱烟味儿听着粗豪沉哑的话语如走入童话境界
语 如 走 yu3 ru2
夏夜在场院乘凉闻着浓烈的旱烟味儿听着粗豪沉哑的话语如走入童话境界
如 走 入 ru2 zou3
夏夜在场院乘凉闻着浓烈的旱烟味儿听着粗豪沉哑的话语如走入童话境界
走 入 童 zou3 ru4
夏夜在场院乘凉闻着浓烈的旱烟味儿听着粗豪沉哑的话语如走入童话境界
入 童 话 ru4 er2
夏夜在场院乘凉闻着浓烈的旱烟味儿听着粗豪沉哑的话语如走入童话境界
童 话 境 tong2 hua4
夏夜在场院乘凉闻着浓烈的旱烟味儿听着粗豪沉哑的话语如走入童话境界
话 境 界 hua4 jing4
夏夜在场院乘凉闻着浓烈的旱烟味儿听着粗豪沉哑的话语如走入童话境界
境 界 jing4 jie4
一路升降蜿蜒如行乱云叠浪之中那九十七米高的华严瀑布更似穿穴于翠丛的玉练
一 路 yi2 yi1
一路升降蜿蜒如行乱云叠浪之中那九十七米高的华严瀑布更似穿穴于翠丛的玉练
九 十 七 shi5 shi2
为吸引游泳爱好者他们发售年卡月卡情侣卡合家欢卡金卡
为 吸 wei4 wei2
为吸引游泳爱好者他们发售年卡

台 组 长 zu2 zu3
总而言之无论哪里人吃一碗蛇和猫或者蛙和鳝鱼于幸福的家庭是决不会有损伤的
吃 一 碗 yi4 yi1
总而言之无论哪里人吃一碗蛇和猫或者蛙和鳝鱼于幸福的家庭是决不会有损伤的
决 不 会 bu2 bu4
其目的与用意显而易见扰乱正常税收秩序别有用心另有图谋
目 的 与 di4 de5
其目的与用意显而易见扰乱正常税收秩序别有用心另有图谋
收 秩 序 zhi4 chuan1
韩秀玲带着同事姐妹们奔向货场一场鏖战战胜了天雨迎来了艳阳而她却被送进了医院
了 艳 阳 yan4 fan1
可是兔场的农民饲养员给兔子喂食喂药都被兔子顺顺当当接受了
顺 顺 当 shun5 shun4
可是兔场的农民饲养员给兔子喂食喂药都被兔子顺顺当当接受了
接 受 了 shou4 xiang3
上午九时随着天津民园体育场内发令枪一声脆响六十一名女运动员蜂拥起跑
枪 一 声 yi4 yi1
上午九时随着天津民园体育场内发令枪一声脆响六十一名女运动员蜂拥起跑
十 一 名 yi4 yi1
上午九时随着天津民园体育场内发令枪一声脆响六十一名女运动员蜂拥起跑
拥 起 跑 qi2 qi3
北京丰台区农民自己花钱筹办万佛延寿寺迎春庙会吸引了区内六十支秧歌队参赛
十 支 秧 zhi1 xin1
北京丰台区农民自己花钱筹办万佛延寿寺迎春庙会吸引了区内六十支秧歌队参赛
秧 歌 队 ge5 ge1
置于厅堂或者楼台只能旁观不可把玩给人以坚稳凝重气宇轩昂的震撼力
置 于 zhi4 zhi5
可别小看这些白糁糁在国际市场一公斤能卖两千多美元该厂一年能生产五吨相当于十几吨黄金
厂 一 年 yi4 yi1
山路陡峭坡度又大村民给车打掩不当车停下不久突然开始顺坡下滑
掩 不 当 bu2 bu4
山路陡峭坡度又大村民给车打掩不当车停下不久突然开始顺坡下滑
不 当 车 dang4 dang1
除家用个体餐饮用冰柜外商业用冷藏冷冻陈列柜等专业用产品也有相当大的市场
用 产 品 chan2 chan3
除家用个体餐饮用冰柜外商业用冷藏冷冻陈列柜等专业用产品也有相当大的市场
品 也 有 ye2 ye3
假日偶有朋友约访我们也多商定于某茶馆践约既省家人应酬又得舒心环境
假 日 jia4 jia3
假日偶有朋友约访我们也多商定于某茶馆践约既省家人应酬又得舒心环境
有 朋 友 peng2 qin1
假日偶有朋友约访我们

分 子 则 zi3 zi5
为了让嫩竹能欺霜傲雪钻天挺拔李宁把虚怀骨硬的竹的精神运用在他的工作之中
为 了 wei4 wei2
为了让嫩竹能欺霜傲雪钻天挺拔李宁把虚怀骨硬的竹的精神运用在他的工作之中
精 神 运 shen5 shen2
厅的窗口被用黑布遮得严严实实厅内昏暗朦胧唯有几串红绿灯频频闪烁
遮 得 严 de5 de2
厅的窗口被用黑布遮得严严实实厅内昏暗朦胧唯有几串红绿灯频频闪烁
严 实 实 shi5 shi2
厅的窗口被用黑布遮得严严实实厅内昏暗朦胧唯有几串红绿灯频频闪烁
实 实 厅 shi5 shi2
厅的窗口被用黑布遮得严严实实厅内昏暗朦胧唯有几串红绿灯频频闪烁
频 频 闪 pin5 pin2
厅的窗口被用黑布遮得严严实实厅内昏暗朦胧唯有几串红绿灯频频闪烁
频 闪 烁 shan3 shan2
郑恩波的刘绍棠传用浓墨重彩活生生地刻画出刘绍棠的为文和为人
棠 传 用 zhuan4 chuan2
郑恩波的刘绍棠传用浓墨重彩活生生地刻画出刘绍棠的为文和为人
生 地 刻 de5 di4
香快燃完了唐老六还没有来接班我心里只骂他混帐要是多有个人也好助我一把
人 也 好 ye2 ye3
祖母绿矿床是由含铍的花岗岩岩浆与富含铬贫二氧化硅的超基性岩相互作用而形成的
祖 母 zu2 zu3
这碗离娘饭姑娘再有离娘痛楚也要每样都吃一点才算循规遵俗的
姑 娘 再 niang5 niang2
这碗离娘饭姑娘再有离娘痛楚也要每样都吃一点才算循规遵俗的
吃 一 点 yi4 yi1
一旦遇到猎物立即用脚须钳住尾巴钩转用尾刺注射一针将猎物毒死
一 旦 yi2 yi1
一旦遇到猎物立即用脚须钳住尾巴钩转用尾刺注射一针将猎物毒死
尾 巴 钩 ba5 ba1
一旦遇到猎物立即用脚须钳住尾巴钩转用尾刺注射一针将猎物毒死
射 一 针 yi4 yi1
一旦遇到猎物立即用脚须钳住尾巴钩转用尾刺注射一针将猎物毒死
将 猎 物 lie4 bao1
于是课堂上拚命演算数学题而将语文政治等丢到一边终于未能进高校大门
拚 命 演 ming4 wan2
于是课堂上拚命演算数学题而将语文政治等丢到一边终于未能进高校大门
到 一 边 yi4 yi1
发射约二十分钟后飞行样机打开降落伞溅落在北纬二十八度五十一分东经一百四十三度四十分的太平洋海面上
落 伞 溅 san3 juan1
发射约二十分钟后飞行样机打开降落伞溅

在 堂 屋 er5 tang2
回到家黄娘娘的奶娃儿正哭得嘶声哑气大一点的一个娃儿在堂屋地上爬
堂 屋 地 zai4 wu1
回到家黄娘娘的奶娃儿正哭得嘶声哑气大一点的一个娃儿在堂屋地上爬
屋 地 上 tang2 di4
回到家黄娘娘的奶娃儿正哭得嘶声哑气大一点的一个娃儿在堂屋地上爬
地 上 爬 wu1 shang4
此消息一出立即轰动了全院一些亲朋故友纷纷出面劝阻高树槐要三思而后行
消 息 一 xi5 xi1
此消息一出立即轰动了全院一些亲朋故友纷纷出面劝阻高树槐要三思而后行
院 一 些 yi4 yi1
此消息一出立即轰动了全院一些亲朋故友纷纷出面劝阻高树槐要三思而后行
亲 朋 故 peng2 qin1
此消息一出立即轰动了全院一些亲朋故友纷纷出面劝阻高树槐要三思而后行
面 劝 阻 quan4 ye3
地壳是内部圈层的最外层由风化的土层和坚硬的岩石组成所以地壳也可称为岩石圈
地 壳 de5 di4
地壳是内部圈层的最外层由风化的土层和坚硬的岩石组成所以地壳也可称为岩石圈
成 所 以 suo2 suo3
中华人民共和国中央军事委员会最近发布命令授予吴孟超模范医学专家荣誉称号
授 予 吴 yu3 yu2
著名演员有成方圆牟炫甫远征张强陈俊华訾莎莉欧阳滨林文增周维等
华 訾 莎 zi1 you3
著名演员有成方圆牟炫甫远征张强陈俊华訾莎莉欧阳滨林文增周维等
莎 莉 欧 li4 li5
美国一些机构往往机关算尽结果是越搅越混越抹越黑最后常常损及自身
国 一 些 yi4 yi1
美国一些机构往往机关算尽结果是越搅越混越抹越黑最后常常损及自身
构 往 往 wang2 wang3
美国一些机构往往机关算尽结果是越搅越混越抹越黑最后常常损及自身
越 混 越 hun2 hun4
这恶魔甩着头发喷着口沫张牙舞爪地扑在老妇人的颈子上扼住她的脆弱的喉管
头 发 喷 fa5 fa1
这恶魔甩着头发喷着口沫张牙舞爪地扑在老妇人的颈子上扼住她的脆弱的喉管
牙 舞 爪 wu2 wu3
这恶魔甩着头发喷着口沫张牙舞爪地扑在老妇人的颈子上扼住她的脆弱的喉管
舞 爪 地 zhao3 zhua3
这恶魔甩着头发喷着口沫张牙舞爪地扑在老妇人的颈子上扼住她的脆弱的喉管
爪 地 扑 de5 di4
这恶魔甩着头发喷着口沫张牙舞爪地扑在老妇人的颈子上扼住她的脆弱的喉管
颈 子 上 zi3 zi5
这恶魔甩着头发

食 可 以 ke2 ke3
幼蛙和成蛙喜吃肉食可以捕捞蝇蛆养殖蚯蚓捕捉害虫等方法为成蛙提供食料
法 为 成 wei4 wei2
幼蛙和成蛙喜吃肉食可以捕捞蝇蛆养殖蚯蚓捕捉害虫等方法为成蛙提供食料
提 供 食 gong4 neng2
家住焦东矿家属院的申老虎由于所在企业效益不佳加上长年患病医疗费已花去二万多元生活极度困难
申 老 虎 lao2 lao3
家住焦东矿家属院的申老虎由于所在企业效益不佳加上长年患病医疗费已花去二万多元生活极度困难
加 上 长 shang5 shang4
家住焦东矿家属院的申老虎由于所在企业效益不佳加上长年患病医疗费已花去二万多元生活极度困难
上 长 年 chang2 zhang3
家住焦东矿家属院的申老虎由于所在企业效益不佳加上长年患病医疗费已花去二万多元生活极度困难
已 花 去 hua1 you3
从使用常温推进到液氢液氧低温高能推进剂并解决了低温技术的应用问题
的 应 用 ying4 ying1
且夫孥孥阿文确尚无偷文如欧阳公之恶德而文章亦较为能做做者也
较 为 能 wei2 wei4
且夫孥孥阿文确尚无偷文如欧阳公之恶德而文章亦较为能做做者也
做 做 者 zuo5 zuo4
要理顺产权关系除了要理顺政企关系外首先要理顺企业资产所有者与经营者的关系
关 系 除 xi5 xi4
要理顺产权关系除了要理顺政企关系外首先要理顺企业资产所有者与经营者的关系
关 系 外 xi5 xi4
要理顺产权关系除了要理顺政企关系外首先要理顺企业资产所有者与经营者的关系
产 所 有 suo2 suo3
要理顺产权关系除了要理顺政企关系外首先要理顺企业资产所有者与经营者的关系
所 有 者 you2 you3
要理顺产权关系除了要理顺政企关系外首先要理顺企业资产所有者与经营者的关系
有 者 与 zhe3 lai2
要理顺产权关系除了要理顺政企关系外首先要理顺企业资产所有者与经营者的关系
关 系 xi5 xi4
到了有庆提着一篮草回来上学也快迟到了急忙往嘴里塞一碗饭边嚼边往城里跑
回 来 上 lai5 lai2
到了有庆提着一篮草回来上学也快迟到了急忙往嘴里塞一碗饭边嚼边往城里跑
往 嘴 里 zui2 zui3
到了有庆提着一篮草回来上学也快迟到了急忙往嘴里塞一碗饭边嚼边往城里跑
塞 一 碗 yi4 yi1
陈云同志同时要求干部认真学习业务精通业务向一切业务内行人

报 复 爱 fu5 fu4
明代的沈周诗书画均自成一家尤工于画与唐寅文征明仇英并称明之四大家
成 一 家 yi4 yi1
基诺族男子穿白色无领棉布上衣衣背后绣有圆形彩色光芒图案下穿棉布白裤
男 子 穿 zi3 zi5
基诺族男子穿白色无领棉布上衣衣背后绣有圆形彩色光芒图案下穿棉布白裤
后 绣 有 xiu4 ye4
基诺族男子穿白色无领棉布上衣衣背后绣有圆形彩色光芒图案下穿棉布白裤
光 芒 图 mang2 fu4
靠卖大碗茶起家后老尹更着迷于洋溢在茶馆中的那浓郁的文化味了
后 老 尹 lao2 lao3
靠卖大碗茶起家后老尹更着迷于洋溢在茶馆中的那浓郁的文化味了
更 着 迷 zhao2 zhe5
中国要求在别国遗弃化学武器的国家应按公约规定尽快彻底销毁所有遗弃化学武器
国 要 求 yao1 yao4
中国要求在别国遗弃化学武器的国家应按公约规定尽快彻底销毁所有遗弃化学武器
毁 所 有 suo2 suo3
两次冬奥会冠军维特为了能够参加本届冬奥会放弃了职业比赛重新转为业余选手
会 冠 军 guan4 guan1
两次冬奥会冠军维特为了能够参加本届冬奥会放弃了职业比赛重新转为业余选手
特 为 了 wei4 wei2
两次冬奥会冠军维特为了能够参加本届冬奥会放弃了职业比赛重新转为业余选手
加 本 届 ben3 ben2
两次冬奥会冠军维特为了能够参加本届冬奥会放弃了职业比赛重新转为业余选手
赛 重 新 chong2 zhong4
两次冬奥会冠军维特为了能够参加本届冬奥会放弃了职业比赛重新转为业余选手
余 选 手 xuan2 xuan3
她没有多少豪言壮语但她却把爱亲人爱家庭爱祖国爱军队完美地统一了起来
美 地 统 de5 di4
她没有多少豪言壮语但她却把爱亲人爱家庭爱祖国爱军队完美地统一了起来
起 来 lai5 lai2
韩电位于西北电网末端其安全生产对该电网稳定运行有重要意义
意 义 yi5 yi4
国外有关专家调查了三百多名长寿者认为勤于用脑可延缓衰老而懒惰可使人早衰
名 长 寿 chang2 zhang3
国外有关专家调查了三百多名长寿者认为勤于用脑可延缓衰老而懒惰可使人早衰
懒 惰 可 duo4 shen4
如今在洱海湖面上三千多艘运输船捕捞船农用和养鱼船日夜穿梭忙碌
面 上 三 shang5 shang4
阿拉法特的夫人苏哈抱着不满五个月的女儿扎哈瓦在耶稣诞生的